In [ ]:
###########import packages##########
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn import ensemble
from sklearn import svm
from sklearn import neighbors
from sklearn import tree
from sklearn.linear_model import BayesianRidge
from sklearn.tree import ExtraTreeRegressor
from sklearn.tree import ExtraTreeClassifier
from sklearn import linear_model
import lightgbm
import catboost
import xgboost
#import shap
#import seaborn as sns
from catboost import *
from sklearn.decomposition import PCA
from sklearn.metrics import roc_curve, auc
#%matplotlib
###########loading data##########
fdata=pd.read_csv('database_filled_ST.csv',encoding="gbk")
raw_data=fdata.loc[:,[
                      'Operating Temperature (℃)',#0
                      'Flow Rate (mL min-1)',#1
                      'Active Area (cm2)', #2
                      'Ir wt. %',#3
                      'Ru wt.%',#4
                      'O wt. %',#5
                      'I/C in Anode',#6
                      'Pt wt. %',#7
                      'I/C in Cathode',#8 
                      'Anode Precious Metal Loading (mg cm-2 Ir/Ru/Pt/Pd)',#9
                      'Cathode Precious Metal Loading (mg cm-2 Pt/Pd)',#10
                      'Membrane Thickness (μm)',#11
                      'EW',#12
                      'Maximum Stability Current Density (A cm-2)',#13
                      'Stability Test Time (h-1)'#14
                        ]]
###########train test splitting##########
raw_param=raw_data.iloc[:,0:15]
print('ready')


In [ ]:
ST50=fdata.loc[:,['CL_50']]

In [ ]:
def summarize(list1):
    total=0
    for ele in range(0, len(list1)):
        total = total + list1[ele]
    return total

In [ ]:
def auc_curve(y_label, y_pre,algorithm_name):
    y_label = y_label + 1
    y_pre = y_pre + 1
    fpr, tpr, thersholds = roc_curve(y_label, y_pre, pos_label=2)
    roc_auc = auc(fpr, tpr)
    x_line=np.arange(0,1.01,0.01)
    y_line=np.arange(0,1.01,0.01)
    print('auc',roc_auc)
    fig=plt.figure()
    plt.plot(fpr, tpr, 'k--', label='ROC (AUC/area = {0:.2f})'.format(roc_auc), lw=2)
    plt.plot(x_line,y_line,c='red')
    plt.xlim([-0.05, 1.05])  # 设置x、y轴的上下限，以免和边缘重合，更好的观察图像的整体
    plt.ylim([-0.05, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')  # 可以使用中文，但需要导入一些库即字体
    plt.title('ROC Curve of %s' %algorithm_name)
    plt.legend(loc="lower right")
    plt.savefig('ROC Curve of %s CL2V2A.png' %algorithm_name)



In [ ]:
def gridsearch_and_return_prediction(model,param,algorithm_name):
    print('start')
    grid = GridSearchCV(model,param_grid=param,cv=5)
    grid.fit(X_train,y_train)
    print('Best Classifier:',grid.best_params_,'Best Score:', grid.best_score_)
    best_model=grid.best_estimator_
    prediction_train=best_model.predict(X_train)
    prediction_test=best_model.predict(X_test)
    final_result=classification_report(y_test,prediction_test,output_dict=True)
    ##############################################################
    y_score=best_model.predict_proba(X_test)
    y_score=y_score[:,1]
    auc_curve(y_test,y_score,algorithm_name)
    ##############################################################
    print(classification_report(y_train,prediction_train))
    print(classification_report(y_test,prediction_test))
    print(final_result['accuracy'])
    ###########generating a figure##########
    print(algorithm_name)
    print(best_model.feature_importances_)
    return prediction_train
def gridsearch_surrogate(model,param,algorithm_name,X_train_surro,y_train_surro):
    print('start')
    grid = GridSearchCV(model,param_grid=param,cv=5)
    grid.fit(X_train_surro,y_train_surro)
    print('Best Classifier:',grid.best_params_,'Best Score:', grid.best_score_)
    best_model=grid.best_estimator_
    prediction_train_surro=best_model.predict(X_train_surro)
    final_result=classification_report(y_train_surro,prediction_train_surro,output_dict=True)
    ##############################################################
    ##############################################################
    print(classification_report(y_train_surro,prediction_train_surro))
    print(final_result['accuracy'])
    return best_model

In [ ]:
seed=230
X_train, X_test, y_train, y_test = train_test_split(raw_param, ST50, test_size=.15,random_state=seed)

In [ ]:
##########CatBoost gridsearch CV for best hyperparameter##########
model_CatClassifier=catboost.CatBoostClassifier(random_state=1,verbose=0)
param_cat = {
'learning_rate':[0.001],
'n_estimators':[50],
"boosting_type":["Plain"],
'max_depth':[7],
'subsample':[0.9],
'reg_lambda':[0]
}
prediction_train=gridsearch_and_return_prediction(model_CatClassifier,param_cat,'CatBoost')

In [ ]:
###########Decision Tree gridsearch CV for best hyperparameter##########
model_DecisionTreeClassifier = tree.DecisionTreeClassifier(random_state=1)
param_DT = {
'max_depth':[3,4,5,6,7,8],
'criterion' : ['gini','entropy'],
'splitter' : [ "best",'random'],
'max_features':['auto','sqrt','log2']
}
DT_model=gridsearch_surrogate(model_DecisionTreeClassifier,param_DT,'Decision Tree',X_train,prediction_train)

In [ ]:
prediction_train

In [ ]:
from dtreeviz.trees import dtreeviz
viz = dtreeviz(DT_model,
               X_train,
               prediction_train,
               target_name='',
               feature_names=raw_param.columns,
               class_names={0:'>= 50μV h-1',1:'< 50μV h-1'},scale=2)
              
viz

In [ ]:
viz.view()